#使用Estimators构建CNN



In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
import os
os.chdir('/content/drive/My Drive/data')
!pwd

/content/drive/My Drive/data


该[tf.layers](https://www.tensorflow.org/api_docs/python/tf/layers)模块提供了一个高级API，可以轻松构建神经网络。它提供了有助于创建密集（完全连接）层和卷积层，添加激活函数以及应用压差正则化的方法。在本教程中，您将学习如何使用layers构建卷积神经网络模型来识别MNIST数据集中的手写数字。

![来自MNIST数据集的手写数字0-9](https://www.tensorflow.org/images/mnist_0-9.png)

所述[MNIST数据集](http://yann.lecun.com/exdb/mnist/)包括60000个训练样例和手写数字0-9，格式为28x28像素的单色图像万个测试实施例。

##入门
让我们为TensorFlow程序设置导入：

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import numpy as np

tf.logging.set_verbosity(tf.logging.INFO)

##卷积神经网络简介
卷积神经网络（CNN）是用于图像分类任务的当前最先进的模型架构。CNN将一系列滤波器应用于图像的原始像素数据，以提取和学习更高级别的特征，然后该模型可用于分类。CNN包含三个组成部分：

卷积层，它将指定数量的卷积滤镜应用于图像。对于每个子区域，图层执行一组数学运算以在输出要素图中生成单个值。然后，卷积层通常将 [ReLU激活函数](https://en.wikipedia.org/wiki/Rectifier_(neural_networks))应用于输出以将非线性引入模型中。

汇集层，对 由卷积层提取的[图像数据](https://en.wikipedia.org/wiki/Convolutional_neural_network#Pooling_layer)进行 [下采样](https://en.wikipedia.org/wiki/Convolutional_neural_network#Pooling_layer)，以减少特征映射的维数，从而减少处理时间。常用的池化算法是最大池化，其提取特征图的子区域（例如，2×2像素块），保持其最大值，并丢弃所有其他值。

密集（完全连接）层，对卷积层提取的特征进行分类，并由池化层进行下采样。在密集层中，层中的每个节点都连接到前一层中的每个节点。

通常，CNN由执行特征提取的一堆卷积模块组成。每个模块由一个卷积层和一个池化层组成。最后一个卷积模块之后是一个或多个执行分类的密集层。CNN中的最终密集层包含模型中每个目标类的单个节点（模型可以预测的所有可能类），使用 [softmax](https://en.wikipedia.org/wiki/Softmax_function)激活函数为每个节点生成0-1之间的值（所有节点的总和）这些softmax值等于1）。我们可以将给定图像的softmax值解释为图像落入每个目标类的可能性的相对测量值。

>注意：有关CNN架构的更全面的演练，请参阅[斯坦福大学的视觉识别卷积神经网络课程资料](https://cs231n.github.io/convolutional-networks/)。
##构建CNN MNIST分类器
让我们构建一个模型，使用以下CNN架构对MNIST数据集中的图像进行分类：

  1. 卷积层＃1：应用32个5x5滤镜（提取5x5像素子区域），具有ReLU激活功能
  2. 池化层＃1：使用2x2过滤器和步长为2执行最大池化（指定池化区域不重叠）
  3. 卷积层＃2：应用64个5x5过滤器，具有ReLU激活功能
  4. 池层＃2：再次，使用2x2过滤器和步幅2执行最大池化
  5. 密集层＃1：1,024个神经元，丢失正则化率为0.4（在训练期间任何给定元素将被丢弃的概率为0.4）
  6. 密集层＃2（Logits Layer）：10个神经元，每个数字目标类一个（0-9）。
  
该[tf.layers](https://www.tensorflow.org/api_docs/python/tf/layers)模块包含创建上述三种图层类型的方法：

  - `conv2d()`。构造一个二维卷积层。将过滤器数量，过滤器内核大小，填充和激活函数作为参数。
  - `max_pooling2d()`。使用max-pooling算法构造二维池化层。将过滤器大小和步幅合并为参数。
  - `dense()`。构造一个密集的层。将神经元数量和激活函数作为参数。
  
这些方法中的每一个都接受张量作为输入并返回变换张量作为输出。这使得将一个层连接到另一个层变得容易：只需从一个层创建方法获取输出并将其作为输入提供给另一个层。

添加以下cnn_model_fn函数，该函数符合TensorFlow的Estimator API所期望的接口（稍后将在[创建估算器中详细介绍](https://www.tensorflow.org/tutorials/estimators/cnn#create_the_estimator)）。此函数采用MNIST特征数据，标签和模式（来自 [tf.estimator.ModeKeys](https://www.tensorflow.org/api_docs/python/tf/estimator/ModeKeys)：TRAIN，EVAL，PREDICT）作为参数; 配置CNN; 并返回预测，损失和培训操作：

In [0]:
def cnn_model_fn(features, labels, mode):
  """Model function for CNN."""
  # Input Layer
  input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

  # Convolutional Layer #1
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #1
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2 and Pooling Layer #2
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

  # Dense Layer
  pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
  dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
  dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits Layer
  logits = tf.layers.dense(inputs=dropout, units=10)

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }

  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])
  }
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

## 培训和评估CNN MNIST分类器
我们编写了MNIST CNN模型函数; 现在我们已准备好进行培训和评估。

###加载培训和测试数据
首先，让我们使用以下代码加载我们的培训和测试数据：

In [6]:
# Load training and eval data 
((train_data, train_labels),(eval_data, eval_labels)) = tf.keras.datasets.mnist.load_data()

train_data = train_data/np.float32(255)
train_labels = train_labels.astype(np.int32) #not required

eval_data = eval_data / np.float32(255)
eval_labels = eval_labels.astype(np.int32) # not required

11493376/11490434 [==============================] - 0s 0us/step


我们存储和训练标记（从0-9的相应值对每个图像）作为训练特征数据（的手绘数字55000个图像的原始像素值）numpy的阵列 中train_data和train_labels分别。同样，我们将评估特征数据（10,000个图像）和评估标签分别存储在eval_data 和中eval_labels。

####创建Estimator {#create_the_estimator}
接下来，让Estimator我们为我们的模型创建一个（用于执行高级模型训练，评估和推理的TensorFlow类）。将以下代码添加到main()：

In [7]:
#Create the Estimator
mnist_classifier = tf.estimator.Estimator(model_fn=cnn_model_fn, 
                                         model_dir="./models/mnist_convnet_model")

I0903 11:58:38.498489 140322995267456 estimator.py:1790] Using default config.
I0903 11:58:38.503556 140322995267456 estimator.py:209] Using config: {'_model_dir': './models/mnist_convnet_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f9f240dad68>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


该model_fn参数指定的模型函数用于培训，评估，和预测; 我们cnn_model_fn在“构建CNN MNIST分类器”中创建了 它。该 model_dir参数指定了模型数据（检查站）将被保存的目录（这里，我们指定的临时目录/tmp/mnist_convnet_model，但随时更改为您选择的另一个目录）。

>注意：有关TensorFlow EstimatorAPI 的深入演练，请参阅tf.estimator中的创建估算器教程。
###设置记录挂钩{#set_up_a_logging_hook}
由于CNN可能需要一段时间来训练，我们设置一些日志记录，以便我们可以在训练期间跟踪进度。我们可以使用TensorFlow tf.train.SessionRunHook创建一个 tf.train.LoggingTensorHook 记录来自CNN的softmax层的概率值。将以下内容添加到main()：

In [0]:
#Set up logging for predictions
tensors_to_log = {"probabilities": "softmax_tensor"}

logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log,
                                         every_n_iter=50)

我们存储了我们想要登录的张量的词典tensors_to_log。每个键都是我们选择的标签，将打印在日志输出中，相应的标签是TensorTensorFlow图中的a的名称。在这里，我们 probabilities可以找到softmax_tensor，我们在生成概率时提供softmax操作的名称cnn_model_fn。

>注意：如果未通过name参数为操作明确指定名称，TensorFlow将指定默认名称。发现应用于操作的名称的几种简单方法是在TensorBoard上可视化您的图形或启用TensorFlow调试器（tfdbg）。

接下来，我们创建LoggingTensorHook，传递tensors_to_log给 tensors参数。我们设置every_n_iter=50，指定在每50个训练步骤后记录概率。

###训练模型
现在我们已经准备好训练我们的模型，我们可以通过创建train_input_fn 和调用train()来完成mnist_classifier。在numpy_input_fn通话中，我们将训练特征数据和标签分别传递给 x（作为dict）和y。我们设置batch_size的100（这意味着该模型将上的100个例子minibatches培养在每一个步骤）。 num_epochs=None表示模型将训练直到达到指定的步数。我们还设置shuffle=True了改组培训数据。然后训练模型一步并记录输出：

In [9]:
#Train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(x={"x":train_data},
                                                   y=train_labels,
                                                   batch_size=100,
                                                   num_epochs=None,
                                                   shuffle=True)

#Train one step and display the probabilties
mnist_classifier.train(input_fn=train_input_fn,
                      steps=1,
                      hooks=[logging_hook])

W0903 11:58:51.176022 140322995267456 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/training_util.py:236: Variable.initialized_value (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
W0903 11:58:51.196891 140322995267456 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_estimator/python/estimator/inputs/queues/feeding_queue_runner.py:62: QueueRunner.__init__ (from tensorflow.python.training.queue_runner_impl) is deprecated and will be removed in a future version.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
W0903 11:58:51.201565 140322995267456 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_estimator/python/estimator/inputs/queues/feeding_functions.py:500: add_

现在 - 不记录每个步骤集steps=1000来训练模型的时间更长，但是在合理的时间内运行这个例子。训练CNN是计算密集型的。要提高模型的准确性，请增加steps传递的数量train()，例如20,000步。

In [10]:
mnist_classifier.train(input_fn=train_input_fn, steps=1000)

I0903 11:59:06.982355 140322995267456 estimator.py:1145] Calling model_fn.
I0903 11:59:07.194478 140322995267456 estimator.py:1147] Done calling model_fn.
I0903 11:59:07.198129 140322995267456 basic_session_run_hooks.py:541] Create CheckpointSaverHook.
I0903 11:59:07.288053 140322995267456 monitored_session.py:240] Graph was finalized.
I0903 11:59:07.304362 140322995267456 saver.py:1280] Restoring parameters from ./models/mnist_convnet_model/model.ckpt-2
I0903 11:59:07.372034 140322995267456 session_manager.py:500] Running local_init_op.
I0903 11:59:07.378553 140322995267456 session_manager.py:502] Done running local_init_op.
I0903 11:59:07.581566 140322995267456 basic_session_run_hooks.py:606] Saving checkpoints for 2 into ./models/mnist_convnet_model/model.ckpt.
I0903 11:59:07.789265 140322995267456 basic_session_run_hooks.py:262] loss = 2.3030148, step = 2
I0903 11:59:09.028916 140322995267456 basic_session_run_hooks.py:692] global_step/sec: 80.638
I0903 11:59:09.034636 140322995267

###评估模型
培训完成后，我们要评估我们的模型，以确定其在MNIST测试集上的准确性。我们调用该evaluate方法，该方法评估我们在eval_metric_ops参数中指定的指标model_fn。将以下内容添加到main()：

In [11]:
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": eval_data},
    y=eval_labels,
    num_epochs=1,
    shuffle=False)

eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

I0903 11:59:31.784337 140322995267456 estimator.py:1145] Calling model_fn.
I0903 11:59:31.913330 140322995267456 estimator.py:1147] Done calling model_fn.
I0903 11:59:31.938645 140322995267456 evaluation.py:255] Starting evaluation at 2019-09-03T11:59:31Z
I0903 11:59:32.034201 140322995267456 monitored_session.py:240] Graph was finalized.
I0903 11:59:32.052234 140322995267456 saver.py:1280] Restoring parameters from ./models/mnist_convnet_model/model.ckpt-1002
I0903 11:59:32.120081 140322995267456 session_manager.py:500] Running local_init_op.
I0903 11:59:32.131962 140322995267456 session_manager.py:502] Done running local_init_op.
I0903 11:59:32.713228 140322995267456 evaluation.py:275] Finished evaluation at 2019-09-03-11:59:32
I0903 11:59:32.714300 140322995267456 estimator.py:2039] Saving dict for global step 1002: accuracy = 0.7202, global_step = 1002, loss = 1.9268304
I0903 11:59:32.760191 140322995267456 estimator.py:2099] Saving 'checkpoint_path' summary for global step 1002: .

{'accuracy': 0.7202, 'loss': 1.9268304, 'global_step': 1002}


为了创建eval_input_fn，我们进行设置num_epochs=1，以便模型在一个时期的数据上评估指标并返回结果。我们还设置 shuffle=False按顺序迭代数据。

###其他资源
要了解有关TensorFlow中TensorFlow Estimators和CNN的更多信息，请参阅以下资源：

  - 在tf.estimator中创建估算器 提供了对TensorFlow Estimator API的介绍。它介绍了如何配置Estimator，编写模型函数，计算损失以及定义训练操作。
  - 高级卷积神经网络介绍了如何使用较低级别的TensorFlow操作构建MNIST CNN分类模型 而无需估计器。